In [1]:
import openseespy.opensees as ops
import opstool as opst
import opstool.vis.plotly as opsvis


In [2]:
# --- Conversión y parámetros en tonf-m ---

cm = 0.01  # m
m = 1.0  # m
kgf = 0.001  # tonf
tonf = 1.0  # tonf
g = 9.81  # m/s² 

# Propiedades del material (en tonf/m²)
E = 15000 * 210**0.5 * kgf / cm**2  # tonf/m²
nu = 0.2
thickness = 10 * cm  # m
rho = (2.4 * tonf / m**3) / g  # tonf·s²/m⁴

# Dimensiones de la placa
Lx = 3.0  # ancho en X (m)
Lz = 9.0  # alto en Z (m)

# Número de divisiones
nx = 6  # número de elementos en X
nz = 18  # número de elementos en Z

dx = Lx / nx
dz = Lz / nz

# Limpiar modelo anterior
ops.wipe()

In [3]:
ops.model('basic', '-ndm', 3, '-ndf', 6)

# --- Crear nodos ---
nodeTag = 1
nodeTags = []  # para guardar índices
for j in range(nz + 1):
    row = []
    for i in range(nx + 1):
        x = i * dx
        y = 0
        z = j * dz
        ops.node(nodeTag, x, y, z)
        row.append(nodeTag)
        nodeTag += 1
    nodeTags.append(row)

# --- Empotramiento en base (z=0) ---
for i in range(nx + 1):
    ops.fix(nodeTags[0][i], 1, 1, 1, 1, 1, 1)

# --- Material y sección ---
ops.section('ElasticMembranePlateSection', 1, E, nu, thickness, rho)

# --- Crear elementos ShellMITC4 ---
elemTag = 1
elemTags = []
for j in range(nz):
    for i in range(nx):
        n1 = nodeTags[j][i]
        n2 = nodeTags[j][i + 1]
        n3 = nodeTags[j + 1][i + 1]
        n4 = nodeTags[j + 1][i]
        ops.element('ShellMITC4', elemTag, n1, n2, n3, n4, 1)
        elemTags.append(elemTag)
        elemTag += 1

# --- Carga vertical: 20 tonf en nodo superior central ---
mid_i = nx // 2
top_node = nodeTags[-1][mid_i]
print(top_node)
ops.timeSeries('Linear', 1)
ops.pattern('Plain', 1, 1)
ops.load(top_node, 20, 0, 0, 0, 0, 0)

130


In [4]:
print(top_node)

130


In [5]:
"""
fig = opsvis.plot_model(show_nodal_loads=True, show_ele_loads=True)
fig.show()
"""

'\nfig = opsvis.plot_model(show_nodal_loads=True, show_ele_loads=True)\nfig.show()\n'

In [6]:
# --- Análisis ---
ops.system('BandGeneral')
ops.numberer('RCM')
ops.constraints('Plain')
ops.integrator('LoadControl', 0.1)
ops.algorithm('Linear')
ops.analysis('Static')


In [7]:
#Modelo de la placa
opsvis.set_plot_props(cmap="emrld_r", point_size=3.0, notebook=True)
fig = opsvis.plot_model(
    odb_tag=1,
    show_bc=True,                   # para ver las condiciones de frontera
    bc_scale=3.0,                   # Escala de las condiciones de frontera 
    show_outline=True)
fig.show(jupyter_backend="jupyterlab") 

OPSTOOL ::  Model data has been saved to .opstool.output/ModelData-1.nc!

In [8]:
# Creación base de datos opstools
ODB = opst.post.CreateODB(odb_tag=1, project_gauss_to_nodes="extrapolate")

for _ in range(10):
    ops.analyze(1)
    ODB.fetch_response_step()
ODB.save_response()

OPSTOOL ::  All responses data with _odb_tag = 1 saved in .opstool.output/RespStepData-1.nc!

In [9]:
# Para visualizar deformaciones nodales
opsvis.set_plot_props(cmap="emrld_r", point_size=3.0, notebook=True)
fig = opsvis.plot_nodal_responses(
    odb_tag=1,                      # tag de la base de datos
    scale= 3.0,
    show_defo=True,
    resp_type="disp",               # tipo de respuesta (deformaciones)
    resp_dof=("UX", "UY", "UZ"),    # grados de libertad a visualizar
    unit_symbol="cm",                # unidad
    unit_factor=100.0,              # factor de conversión de la unidad
    show_bc=True,                   # para ver las condiciones de frontera
    bc_scale=3.0,                   # Escala de las condiciones de frontera
    style="surface"                 # estilo de visualización
)
fig.show()

OPSTOOL ::  Loading response data from .opstool.output/RespStepData-1.nc ...

In [16]:
# Para visualizar deformaciones nodales
opsvis.set_plot_props(cmap="emrld_r", point_size=3.0, notebook=True)
fig = opsvis.plot_nodal_responses_animation(
    odb_tag=1,                      # tag de la base de datos
    scale= 3.0,                     # escala de la deformada 
    show_defo=True,
    show_undeformed=True,
    resp_type="disp",               # tipo de respuesta (deformaciones)
    resp_dof=("UX", "UY", "UZ"),    # grados de libertad a visualizar
    unit_symbol="cm",                # unidad
    unit_factor=100.0,              # factor de conversión de la unidad
    show_bc=True,                   # para ver las condiciones de frontera
    bc_scale=3.0,                   # Escala de las condiciones de frontera
    style="surface"                 # estilo de visualización
)
fig.update_layout(width=1200, height=800)
fig.write_html("animacion_deformaciones.html")
fig.show()

OPSTOOL ::  Loading response data from .opstool.output/RespStepData-1.nc ...

In [11]:
# --- Visualización ---
opsvis.set_plot_props(cmap="rdylbu", point_size=3.0, notebook=True)

plotter = opsvis.plot_unstruct_responses(
    odb_tag=1,
    step="absMax",
    ele_type="Shell",
    resp_type="sectionForcesAtNodes",
    resp_dof="FXY",
    unit_symbol="tonf/m2",      # unidad de los valores de la sección
    show_defo=True,             # para ver la deformada
    show_values=True,           # para ver los valores de la sección
    defo_scale=3.0,             # Escala de la deformada
    show_bc=True,               # para ver las condiciones de frontera
    bc_scale=2.0,               # Escala de las condiciones de frontera
    style="surface"             # estilo de visualización
)

plotter.show(jupyter_backend="jupyterlab")

OPSTOOL ::  Loading response data from .opstool.output/RespStepData-1.nc ...

In [12]:
# --- Visualización ---
opsvis.set_plot_props(cmap="rdylbu", point_size=3.0, notebook=True)

plotter = opsvis.plot_unstruct_responses(
    odb_tag=1,
    ele_type="Shell",
    resp_type="sectionForcesAtNodes",
    resp_dof="FYY",
    unit_symbol="tonf/m2",  # o la unidad que uses
    show_defo=True,     # para ver la deformada
    show_values=True,
    defo_scale=3.0,     # escala de la deformada
    show_bc=True,
    bc_scale=2.0,
    style="surface"     # estilo de visualización
)

plotter.show(jupyter_backend="jupyterlab")

OPSTOOL ::  Loading response data from .opstool.output/RespStepData-1.nc ...

### Aviso
Cualquier operacion con el backend grafico de matplotlib requerirar un restart del server de jupyer

In [13]:
# --- Visualización ---
opsvis.set_plot_props(cmap="rdylbu", point_size=3.0, notebook=True)

plotter = opsvis.plot_unstruct_responses(
    odb_tag=1,
    ele_type="Shell",
    resp_type="StressesAtNodes",
    resp_dof="sigma22",
    unit_symbol="tonf/m2",  # o la unidad que uses
    show_defo=True,     # para ver la deformada
    show_values=True,
    defo_scale=3.0,     # escala de la deformada
    show_bc=True,
    bc_scale=2.0,
    style="surface"     # estilo de visualización
)

plotter.show(jupyter_backend="jupyterlab")

OPSTOOL ::  Loading response data from .opstool.output/RespStepData-1.nc ...

In [14]:
ops.wipe()